In [8]:
from influxdb_client import InfluxDBClient, Point, WriteOptions

### MQTT

In [22]:
import paho.mqtt.client as mqtt

mqtt_username = "mosquitto"
mqtt_password = "mosquitto"
mqtt_host = "18.170.70.157"
mqtt_port = 1883

topic = "test"

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe(topic)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    # print(msg.topic+" "+str(msg.payload))
    print(str(msg.payload.decode('utf-8')))

    # TODO: Take these values and write them to InfluxDB

def on_publish(client, userdata, mid):
    print("mid: "+str(mid))

mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message
mqtt_client.on_publish = on_publish

mqtt_client.username_pw_set(username=mqtt_username, password=mqtt_password)
mqtt_client.connect(mqtt_host, mqtt_port, 60)

# mqtt_client.loop_start()

0

In [10]:
mqtt_client.publish("sensor/123/987/temp", 15.20232, qos=0)


mid: 1


In [24]:
mqtt_client.publish("external/change_protocol", 1, qos=0)


mid: 2


In [126]:
mqtt_client.loop_forever()

mid: 4
mid: 5


KeyboardInterrupt: 

### InfluxDB

In [10]:
# org = "iotclass"
# bucket = "iotdemo"
# token = "Pk9QK7njaI6nRUN287ig926Uw624co31OpfI5A6MyXUKLp_JRc-gQkIyp5IEzDpxoPxA7DogRGFLA0o80ELIVg==" # Does it need $ in front of it?
# host = "130.136.2.70:9999" # With or without http:/, or https://

org = "olestole"
bucket = "testbucket"
token = "luVwheaLHFYWzkIwd48ahXxx_rwuBwM29ixUxf09rRfXmCwX77hBjUOGyuEM3p-P0HUf2cdfYM1iaie-iGJJPw=="
host = "localhost" # With or without http:/, or https://
port = "8086"
url = f"http://{host}:{port}"

In [11]:
# Establish a connection
client = InfluxDBClient(url=url, token=token, org=org)

In [33]:
client.health()

{'checks': [],
 'commit': 'a2f8538837',
 'message': 'ready for queries and writes',
 'name': 'influxdb',
 'status': 'pass',
 'version': 'v2.2.0'}

1432949 seconds
2.25
171.95
169.70
52.92
6949.57
1.13
1.13
0.71
47.88
47.17
16.67
1931.45
0.36
0.36
17.00
16.73
6.43
684.94
188176
412910
330214
7513201
2153456


In [13]:
# Instantiate the WriteAPI and QueryAPI
write_api = client.write_api()
query_api = client.query_api()

In [27]:
import random
n = 100

rHumValues = [round(random.uniform(0., 40.), 2) for _ in range(n)]
rTempValues = [round(random.uniform(0., 40.), 2) for _ in range(n)]

print(rHumValues, rTempValues)

[25.01, 33.12, 10.75, 6.3, 38.99, 7.12, 2.15, 33.85, 28.54, 0.58, 13.1, 5.74, 34.24, 25.65, 5.11, 21.51, 12.26, 23.2, 18.93, 19.79, 17.75, 30.43, 32.45, 10.28, 14.6, 20.87, 31.57, 38.76, 6.11, 32.48, 34.71, 29.88, 28.85, 20.02, 15.93, 35.87, 37.9, 33.35, 5.95, 15.58, 31.27, 26.22, 1.03, 21.37, 34.31, 25.09, 19.81, 29.75, 33.56, 5.33, 12.67, 26.09, 18.31, 35.52, 6.76, 20.73, 31.8, 1.73, 34.58, 8.08, 32.59, 27.54, 36.05, 17.04, 28.36, 11.02, 34.39, 26.77, 20.06, 23.55, 18.01, 39.05, 3.23, 15.49, 6.58, 34.59, 11.12, 21.77, 39.24, 3.55, 32.13, 10.94, 2.16, 23.48, 30.7, 13.45, 33.67, 21.63, 26.34, 21.92, 22.06, 39.18, 15.82, 38.92, 39.64, 3.89, 26.58, 3.04, 32.95, 33.27] [22.42, 23.29, 27.02, 17.56, 13.02, 20.94, 36.44, 36.11, 35.44, 19.7, 30.72, 35.49, 0.09, 32.65, 10.44, 5.84, 23.27, 10.03, 29.08, 7.47, 31.99, 34.93, 29.69, 35.49, 12.97, 10.19, 20.85, 7.86, 37.04, 16.0, 38.62, 22.2, 12.39, 8.97, 33.24, 28.45, 35.62, 3.73, 39.95, 25.91, 19.82, 7.62, 39.17, 7.31, 2.99, 1.17, 18.79, 2.47, 1.

In [30]:
import time

# Create and write the point
# p1 = Point("tempValue").tag("user", "olestole").field("value", 21.34) # field = humidity or temp
# p2 = Point("humValue").tag("user", "olestole").field("value", 31.34) # field = humidity or temp
# write_api.write(bucket=bucket,org=org,record=p1)
# write_api.write(bucket=bucket,org=org,record=p2)

for i in range(n):
    p1 = Point("tempValue").tag("user", "olestole").field("value", rTempValues[i])
    p2 = Point("humValue").tag("user", "olestole").field("value", rHumValues[i])
    write_api.write(bucket=bucket,org=org,record=p1)
    write_api.write(bucket=bucket,org=org,record=p2)
    time.sleep(0.5)

In [29]:
query1 = f'from(bucket: "{bucket}")\
|> range(start: -10m)\
|> filter(fn: (r) => r._measurement == "tempValue")\
|> filter(fn: (r) => r._field == "value")\
|> aggregateWindow(10m, fn: mean))'

query2 = f'from(bucket: "{bucket}")\
|> range(start: -10m)'

# Return the table and print the result - NOT needed for the exercise
result = client.query_api().query(org=org, query=query2)
results = []
for table in result:
    for record in table.records:
        results.append((record.get_value(), record.get_field()))
print(results)

[(12.34, 'value'), (31.34, 'value'), (31.34, 'value'), (31.34, 'value'), (33.27, 'value'), (12.34, 'value'), (21.34, 'value'), (21.34, 'value'), (21.34, 'value'), (25.42, 'value')]
